In [13]:
import pandas as pd
import re
import os
from multiprocessing import Pool, Process, Queue, cpu_count

In [2]:
import xml.etree.ElementTree as ET

In [5]:

def get_file_lists(path):
    file_list=[]
    for dire in os.listdir(path):
        directory = os.path.join(path, dire)
        if os.path.isdir(directory):
            for file in os.listdir(directory):
                if not file.endswith('.xml'): continue
                full = os.path.join(directory, file)
                file_list.append(full)
        if not directory.endswith('.xml'): continue
        fullname = os.path.join(path, directory)
        file_list.append(fullname)
    return file_list

In [10]:
def parse_file(filename):
    tree = ET.parse(filename)
    a={}
#     for elem in tree.iter():
#         if elem.text:
#             elem.text = elem.text.replace("\n","").strip()
#             re.sub(' +',' ',elem.text)
#             if elem.text:
#     a[elem.tag]=elem.text
    for elem in tree.iter():
    if elem.tag == 'download_date':
        a[elem.tag] = elem.text
        break
    return a

In [12]:
def process_files(filelist,q):
    results=[]
    try:
        for file in filelist:
            results.append(parse_file(file))
    except:
        q.put([])
        raise
    q.put(results)


In [17]:
def parallelize_parsing(file_list):

    cpu_cores = cpu_count()
    q = Queue()
    procs = []

    # Split the input file list into sublists according to the number
    # of the available CPU cores
    for i in range(0, cpu_cores):
        sub_list = [file_list[j] for j in range(0, len(file_list)) if j % cpu_cores == i]

        if len(sub_list) > 0:
            p = Process(target=process_files, args=([sub_list, q]))
            p.start()
            procs.append(p)

    # Collect the results
    all_results = []
    for i in range(0, len(procs)):
        all_results.append(q.get())
    return all_results

In [19]:
files = get_file_lists('ct/')
parallelize_parsing(files)

[['email': 'irene_richard@urmc.rochester.edu'}]]